In [2]:
#!pip install kaggle

In [4]:
import kaggle

In [5]:
from kaggle.api.kaggle_api_extended import KaggleApi

In [7]:
api = KaggleApi()

In [8]:
api.authenticate()

In [10]:
api.competition_download_file('sentiment-analysis-on-movie-reviews',
                             'test.tsv.zip', path='./')
api.competition_download_file('sentiment-analysis-on-movie-reviews',
                             'train.tsv.zip', path='./')

100%|████████████████████████████████████████████████████████████████████████████████| 494k/494k [00:01<00:00, 316kB/s]


100%|██████████████████████████████████████████████████████████████████████████████| 1.28M/1.28M [00:01<00:00, 696kB/s]

In [13]:
import zipfile
with zipfile.ZipFile('./test.tsv.zip','r') as zipref:
    zipref.extractall('./')

In [14]:
import zipfile
with zipfile.ZipFile('./train.tsv.zip','r') as zipref:
    zipref.extractall('./')

In [15]:
import pandas as pd

In [19]:
df = pd.read_csv('train.tsv', sep = '\t', engine='python')

In [20]:
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [21]:
#df.drop_duplicates(subset = ['SentenceID'], keep = 'first')

In [22]:
seq_len = 512
num_samples = len(df)

In [23]:
num_samples

156060

In [24]:
from transformers import BertTokenizer

C:\Users\mihir\anaconda3\envs\ml\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading: 100%|███████████████████████████████████████████████████████████████████| 213k/213k [00:00<00:00, 644kB/s]


In [26]:
tokens = tokenizer(df['Phrase'].tolist(), max_length = seq_len,
                   truncation = True, padding = 'max_length',
                   add_special_tokens = True, return_tensors = 'np')

In [27]:
tokens.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [28]:
import numpy as np
with open('movie-xids.npy', 'wb') as f:
    np.save(f, tokens['input_ids'])
with open('movie-xmask.npy', 'wb') as f:
    np.save(f, tokens['attention_mask'])


In [30]:
arr = df['Sentiment'].values

In [31]:
arr.shape

(156060,)

In [32]:
arr.max()+1

5

In [33]:
labels = np.zeros((num_samples, arr.max()+1))
labels.shape

(156060, 5)

In [34]:
labels

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [36]:
labels[np.arange(num_samples), arr] = 1

In [37]:
labels

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]])

In [38]:
with open('movie-labels.npy', 'wb') as f:
    np.save(f, labels)


### Building Tensorflow Input Pipeline

In [3]:
with open('movie-xids.npy', 'rb') as f:
    Xids = np.load(f, allow_pickle = True)
with open('movie-xmask.npy', 'rb') as f:
    Xmask = np.load(f, allow_pickle = True)
with open('movie-labels.npy', 'rb') as f:
    labels = np.load(f, allow_pickle = True)

In [4]:
import tensorflow as tf

In [5]:
import numpy as np

In [6]:
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))

In [7]:
dataset.take(1)

<TakeDataset shapes: ((512,), (512,), (5,)), types: (tf.int32, tf.int32, tf.float64)>

In [8]:
def map_func(input_ids, masks, labels):
    return {'input_ids': input_ids,
            'attention_mask': masks}, labels

In [9]:
dataset = dataset.map(map_func)

In [10]:
dataset.take(1)

<TakeDataset shapes: ({input_ids: (512,), attention_mask: (512,)}, (5,)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>

In [11]:
batch_size = 16

In [12]:
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder = True)

In [13]:
dataset.take(1)

<TakeDataset shapes: ({input_ids: (16, 512), attention_mask: (16, 512)}, (16, 5)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>

In [14]:
split = 0.9

In [15]:
size = int((Xids.shape[0] / batch_size) * split)

In [16]:
size

8778

In [17]:
train_ds = dataset.take(size)
val_ds = dataset.skip(size)

In [18]:
tf.data.experimental.save(train_ds, 'train')
tf.data.experimental.save(val_ds, 'val')

In [19]:
train_ds.element_spec

({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None),
  'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None)},
 TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))

In [20]:
ds = tf.data.experimental.load('train', element_spec = train_ds.element_spec)

In [21]:
from transformers import TFAutoModel

C:\Users\mihir\anaconda3\envs\ml\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
bert = TFAutoModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [23]:
bert.summary()

Model: "tf_bert_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________


In [24]:
import tensorflow as tf

In [25]:
input_ids = tf.keras.layers.Input(shape = (512,), name='input_ids',
                                  dtype = 'int32')
mask = tf.keras.layers.Input(shape = (512,), name='attention_mask',
                                  dtype = 'int32')
#transformer
embeddings = bert.bert(input_ids, attention_mask=mask)[1]

#classifier head
x = tf.keras.layers.Dense(1024, activation  = 'relu')(embeddings)
y= tf.keras.layers.Dense(5, activation = 'softmax', name='outputs')(x)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [26]:
model = tf.keras.Model(inputs=[input_ids, mask], outputs = y)

In [27]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 108310272   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
dense (Dense)                   (None, 1024)         787456      bert[0][1]                   

In [28]:
model.layers[2].trainable = False

In [29]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 108310272   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
dense (Dense)                   (None, 1024)         787456      bert[0][1]                   

In [30]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 5e-5, decay = 1e-6)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics= [acc])

In [31]:
element_spec = ({'input_ids': tf.TensorSpec(shape=(16, 512), dtype=tf.int32, name=None),
  'attention_mask': tf.TensorSpec(shape=(16, 512), dtype=tf.int32, name=None)},
 tf.TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))

In [32]:
train_ds = tf.data.experimental.load('train', element_spec = element_spec)
val_ds = tf.data.experimental.load('val',element_spec = element_spec)

In [33]:
train_ds.take(1)

<TakeDataset shapes: ({input_ids: (16, 512), attention_mask: (16, 512)}, (16, 5)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>

In [35]:
# history = model.fit(
#     train_ds,
#     validation_data = val_ds,
#     epochs = 3)

In [ ]:
model.save('sentiment_model')

### Model Save and Predictions

In [ ]:
model = tf.keras.models.load_model('sentiment_model')

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
def prep_data(text):
    tokens = tokenizer(text, max_length= 512, truncation = True,
                       padding= 'max_length',add_special_tokens = True,
                       return_tensors = 'tf')
    return {'input_ids':tokens['input_ids'],
            'attention_mask' : tokens['attention_mask']}

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
df_test = pd.read_csv('test.tsv' , sep='\t')

In [ ]:
df_test = df_test.drop_duplicates(subset=['SentenceId'], keep = 'first')

In [ ]:
df['Sentiment'] = None

In [ ]:
for i, row in df.iterrows():
    tokens = prep_data(row['Phrase'])
    probs = model.predict(tokens)
    pred = np.argmax(probs)
    df.at[i, 'Sentiment'] = pred

In [ ]:
df.head()